In [2]:
import pandas as pd
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import numpy as np
nltk.download("punkt", quiet=True)

c:\Users\ungdu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True

In [3]:
file_path = r'C:\Users\ungdu\Downloads\Test_Chatgpt\test_data.csv' #import data
data = pd.read_csv(file_path)
data

,Câu hỏi,Câu trả lời
0,Các quả có mùi vị như thế nào?,Quả cam ngon. Quả táo dở. Quả chanh chua. Quả ...
1,Các quả có hình dáng như thế nào?,"Quả cam có hình tròn. Quả táo có hình tròn, hơ..."


In [4]:
import pandas as pd
import nltk
from IPython.display import display

# Tải bộ dữ liệu phân tách câu của NLTK nếu chưa có
nltk.download('punkt')


# Danh sách chứa tất cả các câu đã phân tách
all_sentences = []

# Lặp qua từng dòng của DataFrame
for _, row in data.iterrows():
    # Lấy nội dung cột "Câu trả lời"
    text = row.get("Câu hỏi", "")
    
    # Kiểm tra nếu text không rỗng và là chuỗi
    if text and isinstance(text, str):
        # Chia văn bản thành các câu
        sentences = nltk.sent_tokenize(text)
        
        # Hiển thị từng câu đã tách (chỉ để kiểm tra)
        display(sentences)
        
        # Thêm các câu này vào danh sách tổng hợp
        all_sentences.extend(sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ungdu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Các quả có mùi vị như thế nào?']

['Các quả có hình dáng như thế nào?']

In [5]:
# Tạo DataFrame từ danh sách các câu
sentences_df = pd.DataFrame({'Câu hỏi': all_sentences})

# Lưu DataFrame vào file CSV
output_filename = 'extracted_sentences.csv'
sentences_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"Đã lưu {len(all_sentences)} câu vào file '{output_filename}'.")

Đã lưu 2 câu vào file 'extracted_sentences.csv'.


In [12]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Đọc file CSV chứa các câu đã được tách
sentences_df = pd.read_csv('extracted_sentences.csv', encoding='utf-8-sig')
print(sentences_df)
# Hiển thị số lượng câu đã được tải
print(f"Đã tải {len(sentences_df)} câu từ 'extracted_sentences.csv'.")

                             Câu hỏi
0     Các quả có mùi vị như thế nào?
1  Các quả có hình dáng như thế nào?
Đã tải 2 câu từ 'extracted_sentences.csv'.


In [13]:
# Bước 3: Vector hóa các câu sử dụng TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences_df['Câu hỏi'])

# Hiển thị kích thước ma trận TF-IDF
print(f"Ma trận TF-IDF có kích thước: {tfidf_matrix.shape}")

# Bước 2: Tính độ tương đồng giữa các câu bằng cosine similarity
# Tính ma trận độ tương đồng giữa các câu
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Hiển thị một phần ma trận độ tương đồng
print("Mẫu độ tương đồng giữa các câu:")
print(cosine_sim[:5, :5])

Ma trận TF-IDF có kích thước: (2, 10)
Mẫu độ tương đồng giữa các câu:
[[1.         0.60297482]
 [0.60297482 1.        ]]


In [8]:
# 1. Lưu Ma Trận TF-IDF
# Chuyển đổi ma trận TF-IDF thành DataFrame
# Lưu ý: Nếu số lượng từ vựng (features) lớn, việc lưu trữ có thể tiêu tốn nhiều dung lượng
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Lưu DataFrame TF-IDF vào CSV
tfidf_output_filename = 'tfidf_matrix.csv'
tfidf_df.to_csv(tfidf_output_filename, index=False, encoding='utf-8-sig')

print(f"Đã lưu ma trận TF-IDF vào file '{tfidf_output_filename}'.")

Đã lưu ma trận TF-IDF vào file 'tfidf_matrix.csv'.


In [9]:
# 2. Lưu Ma Trận Cosine Similarity
# Chuyển đổi ma trận cosine similarity thành DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim, index=sentences_df.index, columns=sentences_df.index)

# Lưu DataFrame cosine similarity vào CSV
cosine_sim_output_filename = 'cosine_similarity_matrix.csv'
cosine_sim_df.to_csv(cosine_sim_output_filename, index=True, encoding='utf-8-sig')

print(f"Đã lưu ma trận cosine similarity vào file '{cosine_sim_output_filename}'.")

Đã lưu ma trận cosine similarity vào file 'cosine_similarity_matrix.csv'.


In [10]:
# Bước 4: Gép các câu lại với nhau dựa trên độ tương đồng

# Thiết lập ngưỡng độ tương đồng
SIMILARITY_THRESHOLD = 0.3  # Bạn có thể điều chỉnh ngưỡng này

# Danh sách để lưu các chunk
chunks = []
current_chunk = sentences_df.iloc[0]['Câu hỏi']  # Bắt đầu với câu đầu tiên

for i in range(1, len(sentences_df)):
    prev_sentence = sentences_df.iloc[i - 1]['Câu hỏi']
    current_sentence = sentences_df.iloc[i]['Câu hỏi']
    
    # Vector hóa câu hiện tại và câu trước đó
    prev_vector = vectorizer.transform([prev_sentence])
    current_vector = vectorizer.transform([current_sentence])
    
    # Tính độ tương đồng giữa câu trước và câu hiện tại
    similarity = cosine_similarity(prev_vector, current_vector)[0][0]
    
    print(f"Độ tương đồng giữa câu {i} và câu {i+1}: {similarity:.2f}")
    
    if similarity >= SIMILARITY_THRESHOLD:
        # Gép câu hiện tại vào chunk hiện tại
        current_chunk += ' ' + current_sentence
    else:
        # Thêm chunk hiện tại vào danh sách và bắt đầu chunk mới
        chunks.append(current_chunk)
        current_chunk = current_sentence

# Thêm chunk cuối cùng vào danh sách
chunks.append(current_chunk)

# Tạo DataFrame từ các chunk
chunks_df = pd.DataFrame({'Chunk': chunks})

# Lưu các chunk vào file CSV mới
output_chunk_filename = 'chunked_sentences.csv'
chunks_df.to_csv(output_chunk_filename, index=False, encoding='utf-8-sig')

print(f"Đã lưu {len(chunks)} chunks vào file '{output_chunk_filename}'.")

Độ tương đồng giữa câu 1 và câu 2: 0.60
Đã lưu 1 chunks vào file 'chunked_sentences.csv'.
